In [56]:
from datasets import load_dataset
import faiss
from sentence_transformers.util import semantic_search
from transformers import AutoTokenizer, AutoModel
import torch
import random
import os

dataset = load_dataset("sentence-transformers/squad", split="train")
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
model = AutoModel.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
device = torch.device("cuda")
model.to(device)

In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input, batch_size=64)
    return cls_pooling(model_output)

In [ ]:
embeddings_dataset = dataset.map(lambda x: {
    "question_embedding": get_embeddings(x["question"]).detach().cpu().numpy(),
    "answer_embedding": get_embeddings(x["answers"]["text"]).detach().cpu().numpy()
})

embeddings_dataset.save_to_disk("squad_embeddings")

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

NameError: name 'device' is not defined